TESTデータの読み込み

In [3]:
from sklearn import svm
import numpy as np
import torch
import os

#user define
DINO_OUTPUT_PATH = "experience/5_1_3232_integrated" #DINOのOUTPUTのPATH.experienceに合流予定.
experience_path = "./experience/20230404" #SVMモデル格納のPATH
CropDataset_path = "Crop_Data/crop_test_data64" #色つけ対象データセット

feature_data = torch.load(os.path.join(DINO_OUTPUT_PATH , "testfeat.pth"))
label_data = torch.load(os.path.join(DINO_OUTPUT_PATH, "testlabels.pth"))
feature_data = feature_data.to('cpu').detach().numpy().copy()
label_data  = label_data .to('cpu').detach().numpy().copy()
print(feature_data.shape)
print(label_data .shape)

np.save(os.path.join(DINO_OUTPUT_PATH, "TEST_feature.npy"), feature_data)
np.save(os.path.join(DINO_OUTPUT_PATH, "TEST_label.npy"), label_data)

(65839, 768)
(65839,)


推論データ格納

In [22]:
import numpy as np
import pickle
import os
from tqdm import tqdm

filepaths = np.load(os.path.join(CropDataset_path, "filename_path.npy"))
features = np.load(os.path.join(DINO_OUTPUT_PATH, "TEST_feature.npy"))
labels = np.load(os.path.join(DINO_OUTPUT_PATH, "TEST_label.npy"))

svm_values = masks = np.zeros((len(filepaths), 4))
cls_idxs = np.zeros((len(filepaths)))

if os.path.exists(experience_path)==False:
    os.mkdir(experience_path)

#model load
with open('model/6464nu=0.1/svm_model0.pkl', 'rb') as f:
    day0model = pickle.load(f)
with open('model/6464nu=0.1/svm_model3.pkl', 'rb') as f:
    day3model = pickle.load(f)
with open('model/6464nu=0.1/svm_model5.pkl', 'rb') as f:
    day5model = pickle.load(f)
with open('model/6464nu=0.1/svm_model7.pkl', 'rb') as f:
    day7model = pickle.load(f)

#SVM値を計算
for i, feature in tqdm(enumerate(features)):
    data = feature.reshape(1, -1)
    svm_values[i, 0] = day0model.decision_function(data)
    svm_values[i, 1] = day3model.decision_function(data)
    svm_values[i, 2] = day5model.decision_function(data)
    svm_values[i, 3] = day7model.decision_function(data)

    #クラス配列作成
    cls_idxs[i] = np.argmax(svm_values[i])

print(svm_values.shape)
np.save(os.path.join(experience_path, "svm_value.npy"), svm_values)
np.save(os.path.join(experience_path, "class_ids.npy"), cls_idxs)


25246it [01:57, 215.76it/s]

(25246, 4)


色付け

In [21]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

#color define rgb
red = [255, 0, 0]
blue = [0, 0, 255]
yellow = [255, 255, 0]
orange = [255, 150, 0]
cls_color = [red, blue, yellow, orange]
#print(cls_color[1])


svm_values = np.load(os.path.join(experience_path, "svm_value.npy"))
cls_idxs = np.load(os.path.join(experience_path, "class_ids.npy"))
filepaths = np.load(os.path.join(CropDataset_path, "filename_path.npy"))
cell_mask = np.load(os.path.join(CropDataset_path, "cellpose_output.npy"))

if(os.path.exists(os.path.join(CropDataset_path, "coloring"))):
    os.mkdir(os.path.join(CropDataset_path, "coloring"))


for i, file in tqdm(enumerate(filepaths)):
    #print(filepaths[10])
    img = cv2.imread(filepaths[i])
    mask_anotation = cv2.imread(filepaths[i].replace("image","mask"))
    mask = cell_mask[i]

    #imageの上に塗りつぶす
    #print(np.unique(mask))
    infe_mask = cv2.imread(filepaths[i])
    w, h = mask.shape
    for k in range(w):
        for l in range(h):
            if(mask[k, l]>=1):
                infe_mask[k, l, :] = cls_color[int(cls_idxs[i])]
    #infe_mask = np.where(mask[:, :]>=1,infe_mask[:, :, 0]=255,img[:, :, 0])

    save_folder = os.path.join(os.path.dirname(filepaths[i]).replace("image","coloring"))
    if(os.path.exists(save_folder)==False):
        os.mkdir(save_folder)
    save_path = os.path.join(save_folder,os.path.basename(filepaths[i]))


    cv2.imwrite(save_path, infe_mask)

    ##print(svm_values[i])
    #print(cls_idxs[i])
    #plt.matshow(mask_anotation)
    #plt.matshow(mask)
    #plt.matshow(infe_mask)
    #plt.matshow(img)


21461it [02:54, 123.30it/s]


KeyboardInterrupt: 

In [8]:

print(cls_idxs.shape)

(25246,)
